In [2]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

library("car")
require("lattice")
require("ez")

library("cowplot")
library("ggbeeswarm")

library("nortest")

library("effectsize")

theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))


options(width=240)
library(tidyverse)

In [3]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")


Attaching package: ‘IRdisplay’


The following object is masked from ‘package:xtable’:

    display


Loading required package: papaja

Loading required package: tinytex



In [4]:
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}

In [5]:
AOV4 <- function(DATA,INFO){

    INFO <- paste("`",INFO,"`",sep="")
    DATA_SUMMARY <- DATA %>%
        group_by(cond0) %>%
        summarise(
            N = n(),
            valX_M = mean(valX),
            valX_SD = sd(valX),
            valX_SE = valX_SD / sqrt(N),
            .groups = "drop"
        )    
    DATA_aov <- aov_4(
        formula  <- FORMULA,
        data     <- DATA,
    )    
    DATA_aov_APA <- apa_print(DATA_aov)
    
    DATA_aov_EMM <- emmeans(DATA_aov, specs<-c("cond0","chan0"))
    # DATA_aov_EMM <- emmeans(DATA_aov, specs<-pairwise~cond0|chan0)
    # DATA_aov_EMM <- emmeans(DATA_aov, specs<-pairwise~cond0:chan0)
    DATA_aov_EMM_APA <- apa_print(DATA_aov_EMM)
    
    # DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM)
    # DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM )

    # cont <- "trt.vs.ctrl1"
    # cont <- "trt.vs.ctrl"
    # cont <- "consec"
    # cont <- "poly"
    # cont <- "trt.vs.ctrlk"
    cont <- "pairwise"
    DATA_aov_EMM_PAIRS <- contrast(DATA_aov_EMM, cont, by = c("chan0"))
    
    DATA_aov_EMM_PAIRS_APA <- apa_print(DATA_aov_EMM_PAIRS)    

    return(list("info"      = INFO,
                "summary"   = DATA_SUMMARY,
                "aov"       = DATA_aov,
                "aov_APA"   = DATA_aov_APA,
                "emm"       = DATA_aov_EMM,
                "emm_APA"   = DATA_aov_EMM_APA,
                "pairs"     = DATA_aov_EMM_PAIRS,
                "pairs_APA" = DATA_aov_EMM_PAIRS_APA
               )
          )
}

# Peaks 0

In [6]:
# ifName="./data/JOB_004/exports/df3_peaks3_crude.csv"
ifName="./data/JOB_004/exports/dfZ_peaksZ_crude.csv"

df0 <- read.csv(file = ifName)
df0 <- subset(df0, CHAN_BUND=="" | is.na(CHAN_BUND)  )
df0 <- subset(df0, quest0=="word_set" )

In [507]:
str(df0)


'data.frame':	110592 obs. of  24 variables:
 $ sort       : int  0 1 2 3 4 5 6 7 8 9 ...
 $ SUBJECT    : int  3 3 3 3 4 4 4 4 5 5 ...
 $ SESSION    : int  1 2 3 4 1 2 3 4 1 2 ...
 $ File       : Factor w/ 128 levels "sub-03mjkf_task-lexdec_run-001",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ TIME_WINDOW: int  2 2 2 2 2 2 2 2 2 2 ...
 $ GENDER     : int  1 1 1 1 2 2 2 2 1 1 ...
 $ PEAK       : num  0.761 2.577 0.796 4.444 1.677 ...
 $ ELEC       : Factor w/ 36 levels "AF3","AF4","C1",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ COND_NUM   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ evoked0    : Factor w/ 1 level "evokedZ": 1 1 1 1 1 1 1 1 1 1 ...
 $ quest0     : Factor w/ 1 level "word_set": 1 1 1 1 1 1 1 1 1 1 ...
 $ cond0      : Factor w/ 4 levels "noun_F","noun_M",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ chan0      : Factor w/ 6 levels "LC","LF","LP",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ tmin0      : num  0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 ...
 $ tmax0      : num  0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 ...
 $ 

In [508]:
df1 <- data.frame(df0)

In [509]:
df1 <- subset(df1, cond0 %in% c("noun_F","noun_M","verb_X") )
# df1 <- subset(df1, cond0 %in% c("noun_X","verb_X") )

In [510]:
'%!in%' <- function(x,y)!('%in%'(x,y))

# df1 <- subset(df1, SUB %!in% c("01mkxf","02ppxm","09msxf","15agxf","17makf","24jkxf","26mwxf","31ltxf","33kkxm","37lpxm") )

In [511]:
df1 <- droplevels(df1)
# df1 <- subset(df1, select = -c(evoked0,SES,TASK,RUN,chanX,quest0,CHAN_BUND) )
df1 <- subset(df1, select = c(cond0,chan0,tmin0,tmax0,mode0,valX,SUB) )


In [512]:
str(df1)

'data.frame':	82944 obs. of  7 variables:
 $ cond0: Factor w/ 3 levels "noun_F","noun_M",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ chan0: Factor w/ 6 levels "LC","LF","LP",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ tmin0: num  0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 ...
 $ tmax0: num  0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 ...
 $ mode0: Factor w/ 1 level "pos": 1 1 1 1 1 1 1 1 1 1 ...
 $ valX : num  0.761 2.577 0.796 4.444 1.677 ...
 $ SUB  : Factor w/ 32 levels "03mjkf","04mkkm",..: 1 1 1 1 2 2 2 2 3 3 ...


In [480]:
summary(df1["cond0"])

    cond0      
 noun_F:27648  
 noun_M:27648  
 verb_X:27648  

In [484]:
c(unique(df1["chan0"]))

$chan0
[1] LF RF LC RC LP RP
Levels: LC LF LP RC RF RP

In [485]:
c(unique(df1["tmin0"]))

$tmin0
[1] 0.19 0.35 0.41 0.60 0.11 0.10

https://www.rdocumentation.org/packages/afex/versions/0.27-2/topics/aov_car

In [486]:
# FORMULA  <- valX~(cond0|SUB)
# TEST     <- adjusted("free")

FORMULA  <- valX~(cond0*chan0|SUB)
TEST     <- adjusted("free")

# Produce all possible condition combinations

In [487]:
CONDS <- c()
CONDS <- c(CONDS,str_c("tmin0==",0.1))
CONDS <- c(CONDS,str_c("tmin0==",0.19))
CONDS <- c(CONDS,str_c("tmin0==",0.35))
CONDS <- c(CONDS,str_c("tmin0==",0.41))
CONDS <- c(CONDS,str_c("tmin0==",0.6))
CONDS <- c(CONDS,str_c("tmin0==",0.11))


In [488]:
CONDS

[1] "tmin0==0.1"  "tmin0==0.19" "tmin0==0.35" "tmin0==0.41" "tmin0==0.6"  "tmin0==0.11"

In [489]:
length(CONDS)

[1] 6

## Example condition defining string

In [490]:
IDX=0
CONDS[1]

[1] "tmin0==0.1"

In [491]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); 
data1 <- data1[complete.cases(data1),]; 
done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); 
display_markdown(done1$info); 
display_markdown("### Table"); 
done1$aov_APA$table; 
display_markdown("### Result"); 
display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); 
display_markdown("### Summary (all done1, incl. NANs)"); 
done1$summary; 
display_markdown("### Expected marginal means"); 
done1$emm_APA$table; 
display_markdown("### Paired comparisons for expected marginal means"); 
done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”


# Condition 1 of 6

`tmin0==0.1`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,6.55,1.78,55.07,1.02,.004,.006
2,Chan0,51.58,1.62,50.07,27.11,< .001,.533
3,Cond0 $\times$ Chan0,0.40,4.37,135.46,0.47,.826,.000


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,3.734987,3.280232,0.04832238
noun_M,4608,3.626835,3.245814,0.04781535
verb_X,4608,3.390593,3.125402,0.04604152


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,2.74,"$[2.08$, $3.39]$",8.22,< .001
noun_M_LC,,noun_M,2.64,"$[1.98$, $3.29]$",7.92,< .001
verb_X_LC,,verb_X,2.39,"$[1.74$, $3.05]$",7.19,< .001
noun_F_LF,LF,noun_F,2.03,"$[1.37$, $2.68]$",6.09,< .001
noun_M_LF,,noun_M,2.02,"$[1.36$, $2.67]$",6.06,< .001
verb_X_LF,,verb_X,1.71,"$[1.05$, $2.37]$",5.13,< .001
noun_F_LP,LP,noun_F,6.00,"$[5.34$, $6.66]$",18.02,< .001
noun_M_LP,,noun_M,5.78,"$[5.12$, $6.43]$",17.35,< .001
verb_X_LP,,verb_X,5.55,"$[4.90$, $6.21]$",16.68,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.10,"$[-0.24$, $0.43]$",0.69,.767
LC_noun_F_verb_X,,noun_F - verb_X,0.34,"$[0.01$, $0.68]$",2.42,.043
LC_noun_M_verb_X,,noun_M - verb_X,0.24,"$[-0.09$, $0.58]$",1.72,.199
LF_noun_F_noun_M,LF,noun_F - noun_M,0.01,"$[-0.33$, $0.34]$",0.06,.998
LF_noun_F_verb_X,,noun_F - verb_X,0.32,"$[-0.02$, $0.65]$",2.24,.067
LF_noun_M_verb_X,,noun_M - verb_X,0.31,"$[-0.03$, $0.64]$",2.18,.077
LP_noun_F_noun_M,LP,noun_F - noun_M,0.22,"$[-0.11$, $0.56]$",1.56,.265
LP_noun_F_verb_X,,noun_F - verb_X,0.45,"$[0.11$, $0.78]$",3.15,.005
LP_noun_M_verb_X,,noun_M - verb_X,0.23,"$[-0.11$, $0.56]$",1.59,.253


In [492]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 2 of 6

`tmin0==0.19`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.43,1.89,58.63,1.13,.042,.003
2,Chan0,10.55,1.66,51.54,22.70,< .001,.150
3,Cond0 $\times$ Chan0,1.07,4.40,136.27,0.43,.378,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,4.106013,2.917217,0.04297466
noun_M,4608,4.146643,2.894191,0.04263546
verb_X,4608,3.890248,2.764584,0.04072616


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,3.84,"$[3.14$, $4.55]$",10.81,< .001
noun_M_LC,,noun_M,3.88,"$[3.18$, $4.58]$",10.91,< .001
verb_X_LC,,verb_X,3.63,"$[2.93$, $4.33]$",10.21,< .001
noun_F_LF,LF,noun_F,3.74,"$[3.04$, $4.44]$",10.52,< .001
noun_M_LF,,noun_M,3.76,"$[3.05$, $4.46]$",10.57,< .001
verb_X_LF,,verb_X,3.42,"$[2.72$, $4.12]$",9.62,< .001
noun_F_LP,LP,noun_F,4.12,"$[3.42$, $4.83]$",11.60,< .001
noun_M_LP,,noun_M,4.28,"$[3.57$, $4.98]$",12.02,< .001
verb_X_LP,,verb_X,3.81,"$[3.11$, $4.52]$",10.73,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.04,"$[-0.38$, $0.30]$",-0.25,.965
LC_noun_F_verb_X,,noun_F - verb_X,0.21,"$[-0.13$, $0.55]$",1.47,.310
LC_noun_M_verb_X,,noun_M - verb_X,0.25,"$[-0.09$, $0.59]$",1.72,.200
LF_noun_F_noun_M,LF,noun_F - noun_M,-0.02,"$[-0.36$, $0.32]$",-0.12,.992
LF_noun_F_verb_X,,noun_F - verb_X,0.32,"$[-0.02$, $0.66]$",2.22,.070
LF_noun_M_verb_X,,noun_M - verb_X,0.34,"$[0.00$, $0.68]$",2.35,.052
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.15,"$[-0.49$, $0.19]$",-1.05,.548
LP_noun_F_verb_X,,noun_F - verb_X,0.31,"$[-0.03$, $0.65]$",2.14,.085
LP_noun_M_verb_X,,noun_M - verb_X,0.46,"$[0.12$, $0.80]$",3.18,.005


In [493]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 3 of 6

`tmin0==0.35`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,10.20,1.96,60.78,1.80,< .001,.016
2,Chan0,66.22,2.49,77.05,15.48,< .001,.533
3,Cond0 $\times$ Chan0,0.28,4.62,143.08,0.54,.911,.000


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,-0.05731882,3.568260,0.05256544
noun_M,4608,0.04127613,3.565433,0.05252378
verb_X,4608,0.51578869,3.289514,0.04845912


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,0.43,"$[-0.27$, $1.13]$",1.21,.227
noun_M_LC,,noun_M,0.63,"$[-0.07$, $1.33]$",1.78,.077
verb_X_LC,,verb_X,1.04,"$[0.34$, $1.74]$",2.94,.004
noun_F_LF,LF,noun_F,-2.72,"$[-3.42$, $-2.03]$",-7.70,< .001
noun_M_LF,,noun_M,-2.71,"$[-3.40$, $-2.01]$",-7.65,< .001
verb_X_LF,,verb_X,-2.22,"$[-2.92$, $-1.52]$",-6.29,< .001
noun_F_LP,LP,noun_F,1.55,"$[0.85$, $2.25]$",4.37,< .001
noun_M_LP,,noun_M,1.75,"$[1.05$, $2.45]$",4.94,< .001
verb_X_LP,,verb_X,2.21,"$[1.51$, $2.91]$",6.24,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.20,"$[-0.62$, $0.22]$",-1.13,.494
LC_noun_F_verb_X,,noun_F - verb_X,-0.61,"$[-1.03$, $-0.19]$",-3.45,.002
LC_noun_M_verb_X,,noun_M - verb_X,-0.41,"$[-0.83$, $0.01]$",-2.32,.056
LF_noun_F_noun_M,LF,noun_F - noun_M,-0.02,"$[-0.44$, $0.40]$",-0.11,.994
LF_noun_F_verb_X,,noun_F - verb_X,-0.50,"$[-0.92$, $-0.08]$",-2.83,.014
LF_noun_M_verb_X,,noun_M - verb_X,-0.48,"$[-0.90$, $-0.06]$",-2.72,.019
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.20,"$[-0.62$, $0.22]$",-1.13,.497
LP_noun_F_verb_X,,noun_F - verb_X,-0.66,"$[-1.08$, $-0.24]$",-3.73,.001
LP_noun_M_verb_X,,noun_M - verb_X,-0.46,"$[-0.88$, $-0.04]$",-2.60,.027


In [494]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”


# Condition 4 of 6

`tmin0==0.41`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,11.02,1.66,51.34,1.91,< .001,.016
2,Chan0,57.33,2.39,73.94,15.91,< .001,.504
3,Cond0 $\times$ Chan0,0.61,3.98,123.46,0.76,.654,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,-0.7797538,3.411917,0.05026228
noun_M,4608,-0.7668945,3.448748,0.05080485
verb_X,4608,-0.2514781,3.164568,0.04661848


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,0.00,"$[-0.69$, $0.68]$",-0.01,.992
noun_M_LC,,noun_M,-0.06,"$[-0.75$, $0.62]$",-0.18,.859
verb_X_LC,,verb_X,0.52,"$[-0.17$, $1.20]$",1.49,.139
noun_F_LF,LF,noun_F,-3.32,"$[-4.01$, $-2.64]$",-9.59,< .001
noun_M_LF,,noun_M,-3.46,"$[-4.14$, $-2.78]$",-9.99,< .001
verb_X_LF,,verb_X,-2.84,"$[-3.52$, $-2.15]$",-8.19,< .001
noun_F_LP,LP,noun_F,0.30,"$[-0.38$, $0.99]$",0.88,.380
noun_M_LP,,noun_M,0.52,"$[-0.16$, $1.21]$",1.51,.132
verb_X_LP,,verb_X,0.86,"$[0.18$, $1.55]$",2.50,.014


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.06,"$[-0.37$, $0.48]$",0.32,.944
LC_noun_F_verb_X,,noun_F - verb_X,-0.52,"$[-0.94$, $-0.10]$",-2.89,.012
LC_noun_M_verb_X,,noun_M - verb_X,-0.58,"$[-1.00$, $-0.15]$",-3.22,.004
LF_noun_F_noun_M,LF,noun_F - noun_M,0.14,"$[-0.28$, $0.56]$",0.77,.719
LF_noun_F_verb_X,,noun_F - verb_X,-0.48,"$[-0.91$, $-0.06]$",-2.69,.021
LF_noun_M_verb_X,,noun_M - verb_X,-0.62,"$[-1.04$, $-0.20]$",-3.46,.002
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.22,"$[-0.64$, $0.20]$",-1.22,.440
LP_noun_F_verb_X,,noun_F - verb_X,-0.56,"$[-0.98$, $-0.14]$",-3.12,.006
LP_noun_M_verb_X,,noun_M - verb_X,-0.34,"$[-0.76$, $0.08]$",-1.90,.142


In [495]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”


# Condition 5 of 6

`tmin0==0.6`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,14.19,1.53,47.35,1.87,< .001,.025
2,Chan0,14.84,2.52,78.05,12.21,< .001,.221
3,Cond0 $\times$ Chan0,1.28,3.85,119.40,0.77,.281,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,3.549594,2.726352,0.04016296
noun_M,4608,3.526074,2.777715,0.04091960
verb_X,4608,2.975979,2.546424,0.03751236


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,3.72,"$[3.12$, $4.31]$",12.39,< .001
noun_M_LC,,noun_M,3.66,"$[3.07$, $4.25]$",12.20,< .001
verb_X_LC,,verb_X,3.17,"$[2.57$, $3.76]$",10.56,< .001
noun_F_LF,LF,noun_F,2.33,"$[1.74$, $2.92]$",7.77,< .001
noun_M_LF,,noun_M,2.27,"$[1.68$, $2.86]$",7.57,< .001
verb_X_LF,,verb_X,1.51,"$[0.92$, $2.10]$",5.04,< .001
noun_F_LP,LP,noun_F,3.15,"$[2.55$, $3.74]$",10.49,< .001
noun_M_LP,,noun_M,3.31,"$[2.72$, $3.90]$",11.03,< .001
verb_X_LP,,verb_X,2.77,"$[2.18$, $3.36]$",9.23,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.06,"$[-0.35$, $0.47]$",0.33,.942
LC_noun_F_verb_X,,noun_F - verb_X,0.55,"$[0.14$, $0.96]$",3.16,.005
LC_noun_M_verb_X,,noun_M - verb_X,0.49,"$[0.08$, $0.90]$",2.83,.014
LF_noun_F_noun_M,LF,noun_F - noun_M,0.06,"$[-0.35$, $0.47]$",0.36,.932
LF_noun_F_verb_X,,noun_F - verb_X,0.82,"$[0.41$, $1.23]$",4.71,< .001
LF_noun_M_verb_X,,noun_M - verb_X,0.76,"$[0.35$, $1.17]$",4.36,< .001
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.16,"$[-0.57$, $0.25]$",-0.93,.624
LP_noun_F_verb_X,,noun_F - verb_X,0.38,"$[-0.03$, $0.79]$",2.18,.078
LP_noun_M_verb_X,,noun_M - verb_X,0.54,"$[0.13$, $0.95]$",3.10,.006


In [496]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“More than one observation per cell, aggregating the data using mean (i.e, fun_aggregate = mean)!”


# Condition 6 of 6

`tmin0==0.11`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.18,1.57,48.78,0.88,.781,.000
2,Chan0,64.46,1.66,51.31,18.24,< .001,.595
3,Cond0 $\times$ Chan0,0.61,3.84,119.13,0.59,.652,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,4608,-0.043656924,2.921096,0.04303181
noun_M,4608,-0.012993300,2.994579,0.04411432
verb_X,4608,0.007313765,2.740240,0.04036755


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,-0.53,"$[-1.07$, $0.00]$",-1.96,.052
noun_M_LC,,noun_M,-0.68,"$[-1.22$, $-0.14]$",-2.49,.013
verb_X_LC,,verb_X,-0.60,"$[-1.14$, $-0.06]$",-2.20,.029
noun_F_LF,LF,noun_F,-2.13,"$[-2.67$, $-1.59]$",-7.82,< .001
noun_M_LF,,noun_M,-2.01,"$[-2.55$, $-1.47]$",-7.38,< .001
verb_X_LF,,verb_X,-2.08,"$[-2.62$, $-1.55]$",-7.65,< .001
noun_F_LP,LP,noun_F,2.06,"$[1.52$, $2.59]$",7.55,< .001
noun_M_LP,,noun_M,2.01,"$[1.47$, $2.54]$",7.36,< .001
verb_X_LP,,verb_X,1.99,"$[1.46$, $2.53]$",7.32,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.15,"$[-0.18$, $0.47]$",1.06,.541
LC_noun_F_verb_X,,noun_F - verb_X,0.07,"$[-0.26$, $0.39]$",0.48,.879
LC_noun_M_verb_X,,noun_M - verb_X,-0.08,"$[-0.40$, $0.25]$",-0.57,.834
LF_noun_F_noun_M,LF,noun_F - noun_M,-0.12,"$[-0.45$, $0.20]$",-0.88,.653
LF_noun_F_verb_X,,noun_F - verb_X,-0.05,"$[-0.37$, $0.28]$",-0.35,.935
LF_noun_M_verb_X,,noun_M - verb_X,0.07,"$[-0.25$, $0.40]$",0.53,.857
LP_noun_F_noun_M,LP,noun_F - noun_M,0.05,"$[-0.27$, $0.38]$",0.37,.928
LP_noun_F_verb_X,,noun_F - verb_X,0.06,"$[-0.26$, $0.39]$",0.45,.893
LP_noun_M_verb_X,,noun_M - verb_X,0.01,"$[-0.31$, $0.34]$",0.08,.996
